In [530]:
import os
import sys
import warnings

import numpy as np
import pandas as pd
from mizani.formatters import percent_format
from plotnine import *
from datetime import datetime
from scipy.stats import norm
import statsmodels.api as sm
from statsmodels.regression.linear_model import OLS
import statsmodels.formula.api as smf
from mizani import transforms
from stargazer.stargazer import Stargazer
from sklearn.model_selection import KFold
from IPython.core.display import HTML
from sklearn.metrics import mean_squared_error
import statsmodels.nonparametric.kernel_regression as loess
warnings.filterwarnings("ignore")


In [531]:
# location folders
dirname = "E:/OneDrive - my.cuesta.edu/ceu/ml/Data_Analysis_3/PS1"
data_in = dirname + "/osfstorage-archive/clean/"

In [532]:
data_all = pd.read_csv(data_in + "morg-2014-emp.csv")

In [533]:
print(data_all.columns)

Index(['Unnamed: 0', 'hhid', 'intmonth', 'stfips', 'weight', 'earnwke',
       'uhours', 'grade92', 'race', 'ethnic', 'age', 'sex', 'marital',
       'ownchild', 'chldpres', 'prcitshp', 'state', 'ind02', 'occ2012',
       'class', 'unionmme', 'unioncov', 'lfsr94'],
      dtype='object')


In [534]:
print(np.sort(data_all["occ2012"].unique()))

[  10   20   40   50   60  100  110  120  135  136  137  140  150  160
  205  220  230  300  310  330  340  350  360  410  420  425  430  500
  510  520  530  540  565  600  630  640  650  700  710  725  726  735
  740  800  810  820  830  840  850  860  900  910  930  940  950 1005
 1006 1007 1010 1020 1030 1050 1060 1105 1106 1107 1200 1220 1240 1300
 1310 1320 1340 1350 1360 1400 1410 1420 1430 1440 1450 1460 1500 1510
 1520 1530 1540 1550 1560 1600 1610 1640 1650 1700 1710 1720 1740 1760
 1800 1820 1840 1860 1900 1910 1920 1930 1965 2000 2010 2015 2016 2025
 2040 2050 2060 2100 2105 2145 2160 2200 2300 2310 2320 2330 2340 2400
 2430 2440 2540 2550 2600 2630 2700 2710 2720 2740 2750 2760 2800 2810
 2825 2830 2840 2850 2860 2900 2910 2920 3000 3010 3030 3040 3050 3060
 3110 3140 3150 3160 3200 3210 3220 3230 3245 3250 3255 3256 3258 3260
 3300 3310 3320 3400 3420 3500 3510 3520 3535 3540 3600 3610 3620 3630
 3640 3645 3646 3647 3648 3649 3655 3700 3710 3720 3730 3740 3750 3800
 3820 

In [535]:
# 4820 Securities, commodities, and financial services sales agents
# 5165 Financial clerks, all other
# 0120 Financial managers
# 0800  - 0950

# print(data_all.dtypes)

# Use the combined condition to select the rows
data = data_all.query("occ2012 == 4820 or occ2012 == 5165 or occ2012 == 120 or (occ2012>799 and occ2012<951)")
np.sort(data["occ2012"].unique())


array([ 120,  800,  810,  820,  830,  840,  850,  860,  900,  910,  930,
        940,  950, 4820, 5165], dtype=int64)

In [536]:
data= data[
    [
        "lfsr94",
        "earnwke",
        "uhours",
        "grade92",
        "race",
        "ethnic",
        "age",
        "sex",
        "marital",
        "ownchild",
        "chldpres",
        "prcitshp",
        "state",
        "ind02",
        "occ2012",
        "class",
        "unionmme", 
        "unioncov"]
]
print(data.shape)

(5150, 18)


In [537]:
np.sort(data["ind02"].unique())


array(['** Alcoholic beverages, merchant wholesalers (*4248)',
       '** Construction (23)',
       '** Drugs, sundries, and chemical and allied products, merchant (wholesalerss*4242, 4246)',
       '** Farm product raw materials, merchant wholesalers (*4245)',
       '** Farm supplies, merchant wholesalers (*42491)',
       '** Groceries and related products, merchant wholesalers (*4244)',
       '** Lumber and other construction materials, merchant wholesalers (*4233)',
       '** Machinery, equipment, and supplies, merchant wholesalers (*4238)',
       '** Metals and minerals, except petroleum, merchant wholesalers (*4235)',
       '** Miscellaneous (nondurable goods, merchant wholesalerss*4249 exc. 42491)',
       '** Miscellaneous durable goods, merchant wholesalers (*4239 exc. 42393)',
       '** Motor vehicles, parts and supplies, merchant wholesalers (*4231)',
       '** Petroleum and petroleum products, merchant wholesalers (*4247)',
       '** Professional and commercial equ

In [538]:
for i in data.index:
    data.at[i,"ind02"] = data.at[i,"ind02"].replace("*", "")
    data.at[i,"ind02"] = data.at[i,"ind02"].replace(" ", "")
np.sort(data["ind02"].unique())


array(['3095', '3365', '3895', '4195', '4265', '4795', '5275',
       'Accounting,taxpreparation,bookkeeping,andpayrollservices(5412)',
       'Administrationofeconomicprogramsandspaceresearch(926,927)',
       'Administrationofenvironmentalqualityandhousingprograms(924,925)',
       'Administrationofhumanresourceprograms(923)',
       'Advertisingandrelatedservices(5418)',
       'Aerospaceproductsandpartsmanufacturing(336414,336415,336419)',
       'Agriculturalchemicalmanufacturing(3253)',
       'Agriculturalimplementmanufacturing(33311)',
       'Aircraftandpartsmanufacturing(336411to336413)',
       'Airtransportation(481)',
       'Alcoholicbeverages,merchantwholesalers(4248)',
       'Aluminumproductionandprocessing(3313)',
       'Animalfood,grainandoilseedmilling(3111,3112)',
       'Animalproduction(112)', 'Animalslaughteringandprocessing(3116)',
       'Architectural,engineering,andrelatedservices(5413)',
       'Automobiledealers(4411)',
       'Automotiveequipmentrentalan

In [539]:
for i in data.index:
    data.at[i, "ind01"] = data.at[i, "ind02"][0:3]

In [540]:
dummies = pd.get_dummies(data["ind01"], prefix="ind01", drop_first=True)

In [541]:
data = pd.concat([data, dummies], axis=1)

In [542]:
data.columns


Index(['lfsr94', 'earnwke', 'uhours', 'grade92', 'race', 'ethnic', 'age',
       'sex', 'marital', 'ownchild',
       ...
       'ind01_Tob', 'ind01_Toy', 'ind01_Tra', 'ind01_Tru', 'ind01_Vet',
       'ind01_War', 'ind01_Was', 'ind01_Wat', 'ind01_Who', 'ind01_Wir'],
      dtype='object', length=161)

In [543]:
data["wage"] = data["earnwke"] / data["uhours"]


In [544]:
data["ln_wage"] = np.log(data["wage"])

In [545]:
data["age2"] = np.power(data["age"], 2)

In [546]:
print(np.sort(data["grade92"].unique()))

[31 32 34 35 36 37 38 39 40 41 42 43 44 45 46]


In [547]:
dict = {31:0, 32: (1+2+3+4)/4, 34:(7+8)/2, 35:9, 36:10, 37:11, 38:12, 39:13, 40:14, 41:15, 42:15, 43:16, 44:18, 45:23, 46:24,}
data=data.replace({"grade92": dict})
print(np.sort(data["grade92"].unique()))

[ 0.   2.5  7.5  9.  10.  11.  12.  13.  14.  15.  16.  18.  23.  24. ]


In [548]:
print(np.sort(data["race"].unique()))

[ 1  2  3  4  5  6  7  8  9 10 11 13 16 19 21]


In [549]:
dummies = pd.get_dummies(data["race"], prefix="race", drop_first=True)
data = pd.concat([data, dummies], axis=1)

In [550]:
dict = {1:1, 2: 0}
data=data.replace({"sex": dict})
print(np.sort(data["sex"].unique()))

[0 1]


In [551]:
print(np.sort(data["marital"].unique()))

[1 2 3 4 5 6 7]


In [552]:
dummies = pd.get_dummies(data["marital"], prefix="marital", drop_first=True)
data = pd.concat([data, dummies], axis=1)

In [553]:
dict = {"No":0, "Yes": 1}
data=data.replace({"unioncov": dict})
print(data["unioncov"].unique())
print(data["unioncov"].unique())
print(data.shape)
na_count = data.isna().sum()
print(na_count.to_list())
# drop unioncov and ethnic => many NaNs => so not using unionmme also
data = data.drop(["ethnic", "unioncov", "unionmme"],axis=1)
na_count = data.isna().sum()
print(na_count.to_list())

[ 0. nan  1.]
[ 0. nan  1.]
(5150, 184)
[0, 0, 0, 0, 0, 4763, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 209, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [554]:
print(data["ownchild"].unique())

[0 1 2 3 4 5 6]


In [555]:
print(data["lfsr94"].unique())

['Employed-At Work' 'Employed-Absent']


In [556]:
dict = {"Employed-At Work":1, "Employed-Absent": 0}
data=data.replace({"lfsr94": dict})
print(data["lfsr94"].unique())

[1 0]


In [557]:
print(data["class"].unique())

['Private, For Profit' 'Private, Nonprofit' 'Government - State'
 'Government - Local' 'Government - Federal']


In [558]:
dummies = pd.get_dummies(data["class"], prefix="class", drop_first=True)
data = pd.concat([data, dummies], axis=1)

In [559]:
print(data["occ2012"].unique())
dummies = pd.get_dummies(data["occ2012"], prefix="occ2012", drop_first=True)
data = pd.concat([data, dummies], axis=1)

[ 120  800  910  850  820 4820  840  860 5165  810  940  950  900  930
  830]


In [560]:
print(data["state"].unique())
dummies = pd.get_dummies(data["state"], prefix="state", drop_first=True)
data = pd.concat([data, dummies], axis=1)

['63' '94' '86' '71' '93' '84' 'CT' '51' '53' '59' '58' '95' '82' 'IL'
 'IN' 'IA' '47' '61' '72' 'ME' '52' 'MA' 'MI' 'MN' '64' 'MO' '81' '46'
 '88' 'NH' 'NJ' '85' 'NY' '56' 'ND' 'OH' '73' '92' 'PA' 'RI' '57' 'SD'
 '62' '74' '87' 'VT' '54' '91' 'WI' '83' '55' '6' '9' '8' '7' '1' '5' '3'
 '4' '2' 4 6 7 1 5 3 8 2 9]


In [561]:
print(data["prcitshp"].unique())
dummies = pd.get_dummies(data["prcitshp"], prefix="prcitshp", drop_first=True)
data = pd.concat([data, dummies], axis=1)

['Native, Born In US' 'Native, Born Abroad Of US Parent(s)'
 'Foreign Born, US Cit By Naturalization'
 'Native, Born in PR or US Outlying Area' 'Foreign Born, Not a US Citizen']


In [562]:
data["exp"] = data["age"] - data["grade92"] - 6
data["exp2"] = data["exp"]*data["exp"]

In [563]:
data.columns.to_list()

['lfsr94',
 'earnwke',
 'uhours',
 'grade92',
 'race',
 'age',
 'sex',
 'marital',
 'ownchild',
 'chldpres',
 'prcitshp',
 'state',
 'ind02',
 'occ2012',
 'class',
 'ind01',
 'ind01_336',
 'ind01_389',
 'ind01_419',
 'ind01_426',
 'ind01_479',
 'ind01_527',
 'ind01_Acc',
 'ind01_Adm',
 'ind01_Adv',
 'ind01_Aer',
 'ind01_Agr',
 'ind01_Air',
 'ind01_Alc',
 'ind01_Alu',
 'ind01_Ani',
 'ind01_Arc',
 'ind01_Aut',
 'ind01_Bak',
 'ind01_Ban',
 'ind01_Bee',
 'ind01_Bev',
 'ind01_Boo',
 'ind01_Bui',
 'ind01_Bus',
 'ind01_Car',
 'ind01_Cem',
 'ind01_Chi',
 'ind01_Civ',
 'ind01_Clo',
 'ind01_Coa',
 'ind01_Col',
 'ind01_Com',
 'ind01_Con',
 'ind01_Cou',
 'ind01_Cro',
 'ind01_Cut',
 'ind01_Dai',
 'ind01_Dat',
 'ind01_Dep',
 'ind01_Dri',
 'ind01_Dru',
 'ind01_Ele',
 'ind01_Emp',
 'ind01_Eng',
 'ind01_Exe',
 'ind01_Far',
 'ind01_Fis',
 'ind01_Foo',
 'ind01_For',
 'ind01_Fou',
 'ind01_Fru',
 'ind01_Fue',
 'ind01_Fun',
 'ind01_Fur',
 'ind01_Gas',
 'ind01_Gif',
 'ind01_Gla',
 'ind01_Gro',
 'ind01_Har',


In [564]:
data_r = data.drop(["wage", "earnwke","ind01","ind02", "state", "class", "occ2012", "prcitshp", "marital", "race"],axis=1)
print(data_r.shape)

data_r["child"] = data_r["ownchild"] * data_r["chldpres"]
data_r["agexp"] = data_r["age"] * data_r["exp"]
data_r["sex_child"] = data_r["sex"] * data_r["child"]


out_sample = data_r.sample(n=250)
data_r = data_r.drop(out_sample.index)
print(data_r.shape)
data_r_X = data_r.drop(["ln_wage"],axis=1)
y = data_r["ln_wage"]
# data_r.dropna(inplace=True)

(5150, 263)
(4900, 266)


In [565]:
data_r.columns.to_list()

['lfsr94',
 'uhours',
 'grade92',
 'age',
 'sex',
 'ownchild',
 'chldpres',
 'ind01_336',
 'ind01_389',
 'ind01_419',
 'ind01_426',
 'ind01_479',
 'ind01_527',
 'ind01_Acc',
 'ind01_Adm',
 'ind01_Adv',
 'ind01_Aer',
 'ind01_Agr',
 'ind01_Air',
 'ind01_Alc',
 'ind01_Alu',
 'ind01_Ani',
 'ind01_Arc',
 'ind01_Aut',
 'ind01_Bak',
 'ind01_Ban',
 'ind01_Bee',
 'ind01_Bev',
 'ind01_Boo',
 'ind01_Bui',
 'ind01_Bus',
 'ind01_Car',
 'ind01_Cem',
 'ind01_Chi',
 'ind01_Civ',
 'ind01_Clo',
 'ind01_Coa',
 'ind01_Col',
 'ind01_Com',
 'ind01_Con',
 'ind01_Cou',
 'ind01_Cro',
 'ind01_Cut',
 'ind01_Dai',
 'ind01_Dat',
 'ind01_Dep',
 'ind01_Dri',
 'ind01_Dru',
 'ind01_Ele',
 'ind01_Emp',
 'ind01_Eng',
 'ind01_Exe',
 'ind01_Far',
 'ind01_Fis',
 'ind01_Foo',
 'ind01_For',
 'ind01_Fou',
 'ind01_Fru',
 'ind01_Fue',
 'ind01_Fun',
 'ind01_Fur',
 'ind01_Gas',
 'ind01_Gif',
 'ind01_Gla',
 'ind01_Gro',
 'ind01_Har',
 'ind01_Hea',
 'ind01_Hom',
 'ind01_Hos',
 'ind01_Hou',
 'ind01_Ind',
 'ind01_Ins',
 'ind01_Int',


In [576]:
columns = data_r.columns.to_list()
state_all = filter(lambda x: x.startswith('state_'), columns)
ind_all = filter(lambda x: x.startswith('ind01_'), columns)
ethnic_all = filter(lambda x: x.startswith('ethnic_'), columns)
occ_all = filter(lambda x: x.startswith('occ2012_'), columns)
class_all = filter(lambda x: x.startswith('class__'), columns)
# data_r.dropna(inplace=True)
# data_r_X.dropna(inplace=True)

In [577]:
metrics = {}
# Fit the OLS model
formula_1 = ["sex", "grade92", "exp2", "lfsr94", "age", "age2"]
formula_2 = ["sex", "grade92", "exp2", "lfsr94", "age", "exp", 'ownchild', 'chldpres']
formula_3 = ["sex", "grade92", "exp2", "lfsr94", "age", "exp", 'ownchild', 'chldpres', "child", "agexp", "sex_child"]
formula_4 = list(ethnic_all) +list(occ_all) + list(ind_all) + list(state_all)+list(class_all)+["sex", "grade92", "exp2", "lfsr94", "age", "exp", 'ownchild', 'chldpres']
def full_ols(formula, name, datas):
    model = OLS(datas['ln_wage'],datas[formula])
    results = model.fit()
    # print(x.columns.to_list())
    # # Print the results
    metrics[name] = {"RMSE":np.sqrt(results.mse_resid) , "BIC": results.bic}
#     print(metrics)
#     print(results.summary())
full_ols(formula_1, "model_1", data_r)
full_ols(formula_2, "model_2", data_r)
full_ols(formula_3, "model_3", data_r)
full_ols(formula_4, "model_4", data_r)

print(metrics)

{'model_1': {'RMSE': 0.5797721448618696, 'BIC': 8608.39884994926}, 'model_2': {'RMSE': 0.5802302632501348, 'BIC': 8631.130575035597}, 'model_3': {'RMSE': 0.5796944450608913, 'BIC': 8644.561650706355}, 'model_4': {'RMSE': 0.5686784926853979, 'BIC': 10107.714984399634}}


In [578]:
k = KFold(n_splits=4, shuffle=True, random_state=2023)

In [579]:
rmse_results = []
for train_index, test_index in k.split(data_r_X):
    # Get the training and test data for this fold
    X_train, X_test = data_r_X.iloc[train_index], data_r_X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    formula = X_train[["sex", "grade92", "exp2", "lfsr94", "age", "age2"]]
    y_train = list(y_train)
    model = OLS(y_train,formula)
    results = model.fit()
    # Predict on the test data
    y_pred = results.predict(X_test[["sex", "grade92", "exp2", "lfsr94", "age", "age2"]])

    # Calculate the RMSE for this fold
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    rmse_results.append(rmse)
metrics["model_1_k"] = {"RMSE_1":rmse_results[0], "RMSE_2":rmse_results[1], "RMSE_3":rmse_results[2], "RMSE_4":rmse_results[3], "RMSE_average":np.mean(rmse_results)}

rmse_results = []
for train_index, test_index in k.split(data_r_X):
#     data_r_X.dropna(inplace=True)
    # Get the training and test data for this fold
    X_train, X_test = data_r_X.iloc[train_index], data_r_X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    formula = X_train[["sex", "grade92", "exp2", "lfsr94", "age", "exp", "ownchild", "chldpres"]]
    y_train = list(y_train)
    model = OLS(y_train,formula)
    results = model.fit()
    # Predict on the test data
    y_pred = results.predict(X_test[["sex", "grade92", "exp2", "lfsr94", "age", "exp", "ownchild", "chldpres"]])

    # Calculate the RMSE for this fold
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    rmse_results.append(rmse)
metrics["model_2_k"] = {"RMSE_1":rmse_results[0], "RMSE_2":rmse_results[1], "RMSE_3":rmse_results[2], "RMSE_4":rmse_results[3], "RMSE_average":np.mean(rmse_results)}

rmse_results = []
for train_index, test_index in k.split(data_r_X):
#     data_r_X.dropna(inplace=True)
    # Get the training and test data for this fold
    X_train, X_test = data_r_X.iloc[train_index], data_r_X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    formula = X_train[list(ind_all) + list(state_all)+list(class_all)+["sex", "grade92", "exp2", "lfsr94", "age", "exp", 'ownchild', 'chldpres', "child", "agexp", "sex_child"]]
    y_train = list(y_train)
    model = OLS(y_train,formula)
    results = model.fit()
    # Predict on the test data
    y_pred = results.predict(X_test[list(ind_all) + list(state_all)+list(class_all)+["sex", "grade92", "exp2", "lfsr94", "age", "exp", 'ownchild', 'chldpres', "child", "agexp", "sex_child"]])

    # Calculate the RMSE for this fold
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    rmse_results.append(rmse)
metrics["model_3_k"] = {"RMSE_1":rmse_results[0], "RMSE_2":rmse_results[1], "RMSE_3":rmse_results[2], "RMSE_4":rmse_results[3], "RMSE_average":np.mean(rmse_results)}

rmse_results = []
for train_index, test_index in k.split(data_r_X):
#     data_r_X.dropna(inplace=True)
    # Get the training and test data for this fold
    X_train, X_test = data_r_X.iloc[train_index], data_r_X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    formula = X_train[list(ind_all) + list(state_all)+list(class_all)+["sex", "grade92", "exp2", "lfsr94", "age", "exp", 'ownchild', 'chldpres',]]
    y_train = list(y_train)
    model = OLS(y_train,formula)
    results = model.fit()
    # Predict on the test data
    y_pred = results.predict(X_test[list(ind_all) + list(state_all)+list(class_all)+["sex", "grade92", "exp2", "lfsr94", "age", "exp", 'ownchild', 'chldpres',]])

    # Calculate the RMSE for this fold
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    rmse_results.append(rmse)
metrics["model_4_k"] = {"RMSE_1":rmse_results[0], "RMSE_2":rmse_results[1], "RMSE_3":rmse_results[2], "RMSE_4":rmse_results[3], "RMSE_average":np.mean(rmse_results)}
print(metrics)

{'model_1': {'RMSE': 0.5797721448618696, 'BIC': 8608.39884994926}, 'model_2': {'RMSE': 0.5802302632501348, 'BIC': 8631.130575035597}, 'model_3': {'RMSE': 0.5796944450608913, 'BIC': 8644.561650706355}, 'model_4': {'RMSE': 0.5686784926853979, 'BIC': 10107.714984399634}, 'model_1_k': {'RMSE_1': 0.610128555071661, 'RMSE_2': 0.6253558210232234, 'RMSE_3': 0.48502984024864093, 'RMSE_4': 0.5898212474504917, 'RMSE_average': 0.5775838659485043}, 'model_2_k': {'RMSE_1': 0.6101285592128982, 'RMSE_2': 0.6272493981270137, 'RMSE_3': 0.4840046558196684, 'RMSE_4': 0.5899945613957996, 'RMSE_average': 0.577844293638845}, 'model_3_k': {'RMSE_1': 0.6095833006287518, 'RMSE_2': 0.6268141252025254, 'RMSE_3': 0.4846398712668718, 'RMSE_4': 0.5891058706835516, 'RMSE_average': 0.5775357919454251}, 'model_4_k': {'RMSE_1': 0.6101285592128982, 'RMSE_2': 0.6272493981270137, 'RMSE_3': 0.4840046558196684, 'RMSE_4': 0.5899945613957996, 'RMSE_average': 0.577844293638845}}


In [582]:
full_ols(formula_1, "model_out_1", out_sample)
full_ols(formula_1, "model_out_2", out_sample)
full_ols(formula_1, "model_out_3", out_sample)
full_ols(formula_4, "model_out_4", out_sample)
print(metrics)

{'model_1': {'RMSE': 0.5797721448618696, 'BIC': 8608.39884994926}, 'model_2': {'RMSE': 0.5802302632501348, 'BIC': 8631.130575035597}, 'model_3': {'RMSE': 0.5796944450608913, 'BIC': 8644.561650706355}, 'model_4': {'RMSE': 0.5686784926853979, 'BIC': 10107.714984399634}, 'model_1_k': {'RMSE_1': 0.610128555071661, 'RMSE_2': 0.6253558210232234, 'RMSE_3': 0.48502984024864093, 'RMSE_4': 0.5898212474504917, 'RMSE_average': 0.5775838659485043}, 'model_2_k': {'RMSE_1': 0.6101285592128982, 'RMSE_2': 0.6272493981270137, 'RMSE_3': 0.4840046558196684, 'RMSE_4': 0.5899945613957996, 'RMSE_average': 0.577844293638845}, 'model_3_k': {'RMSE_1': 0.6095833006287518, 'RMSE_2': 0.6268141252025254, 'RMSE_3': 0.4846398712668718, 'RMSE_4': 0.5891058706835516, 'RMSE_average': 0.5775357919454251}, 'model_4_k': {'RMSE_1': 0.6101285592128982, 'RMSE_2': 0.6272493981270137, 'RMSE_3': 0.4840046558196684, 'RMSE_4': 0.5899945613957996, 'RMSE_average': 0.577844293638845}, 'model_out_1': {'RMSE': 0.44533331502714896, 'BIC

In [585]:
table = pd.concat(
    [
        pd.DataFrame(data=[[metrics["model_1"]["RMSE"], metrics["model_2"]["RMSE"], metrics["model_3"]["RMSE"], metrics["model_4"]["RMSE"]],
[metrics["model_1"]["BIC"], metrics["model_2"]["BIC"], metrics["model_3"]["BIC"], metrics["model_4"]["BIC"]],
[metrics["model_1_k"]["RMSE_1"], metrics["model_2_k"]["RMSE_1"], metrics["model_3_k"]["RMSE_1"], metrics["model_4_k"]["RMSE_1"]],
[metrics["model_1_k"]["RMSE_2"], metrics["model_2_k"]["RMSE_2"],metrics["model_3_k"]["RMSE_2"],metrics["model_4_k"]["RMSE_2"]],
[metrics["model_1_k"]["RMSE_3"], metrics["model_2_k"]["RMSE_3"],metrics["model_3_k"]["RMSE_3"],metrics["model_4_k"]["RMSE_3"]],
[metrics["model_1_k"]["RMSE_4"], metrics["model_2_k"]["RMSE_4"],metrics["model_3_k"]["RMSE_4"],metrics["model_4_k"]["RMSE_4"]],
[metrics["model_1_k"]["RMSE_average"], metrics["model_2_k"]["RMSE_average"],metrics["model_3_k"]["RMSE_average"],metrics["model_4_k"]["RMSE_average"]],
[metrics["model_out_1"]["RMSE"], metrics["model_out_2"]["RMSE"],metrics["model_out_3"]["RMSE"],metrics["model_out_4"]["RMSE"]],
[metrics["model_out_1"]["BIC"], metrics["model_out_2"]["BIC"],metrics["model_out_3"]["BIC"],metrics["model_out_4"]["BIC"]]

                          ],
            columns=["Model 1", "Model 2", "Model 3", "Model 4"]).rename({0: "RMSE - full sample", 1: "BIC - full sample", 2:"Fold 1", 3:"Fold 2", 4:"Fold 3", 5:"Fold 4", 6:"Average RMSE", 7:"Out sample RMSE", 8:"Out sample BIC"}, axis=0),
    ]
).round(3)
print(table)

                     Model 1   Model 2   Model 3    Model 4
RMSE - full sample     0.580     0.580     0.580      0.569
BIC - full sample   8608.399  8631.131  8644.562  10107.715
Fold 1                 0.610     0.610     0.610      0.610
Fold 2                 0.625     0.627     0.627      0.627
Fold 3                 0.485     0.484     0.485      0.484
Fold 4                 0.590     0.590     0.589      0.590
Average RMSE           0.578     0.578     0.578      0.578
Out sample RMSE        0.445     0.445     0.445      0.427
Out sample BIC       332.059   332.059   332.059    760.695
